In [ ]:
%pip install torch==1.11.0+cu113 torchdata==0.3.0 torchtext==0.12.0 -f https://download.pytorch.org/whl/cu113/torch_stable.html
%pip install spacy tqdm
!python -m spacy download en_core_web_sm

In [3]:
import torch
import torchtext
SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

c:\Users\liamd\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version:  2.2.0+cpu
torchtext Version:  0.16.2+cpu
Using CPU.


In [9]:
#load the dataset
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-CW")


In [ ]:
train_data = dataset['train']


# Start by removing stop words with spaCy stopword list

In [12]:
import spacy

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Function to remove stopwords and corresponding labels
def remove_stopwords(tokens, labels):
    # Create a list to store the filtered tokens and labels
    filtered_tokens = []
    filtered_labels = []
    
    # Iterate over the tokens and labels
    for token, label in zip(tokens, labels):
      # Check if the token is a stopword
      current_tokens=[]
      current_labels =[]
      for sub_token, sub_label in zip(token, label):
        if not nlp.vocab[sub_token].is_stop:
          # If not a stopword, add the token and label to the filtered lists
          current_tokens.append(sub_token)
          current_labels.append(sub_label)  
    
    return filtered_tokens, filtered_labels




[['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.']]
[['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']]


In [ ]:
#filter out stopwords from training tokens and labels
filtered_tokens, filtered_labels = remove_stopwords(train_data['tokens'], train_data['labels']) 
train_data['tokens'] = filtered_tokens 
train_data['ner_tags'] = filtered_labels

In [5]:
from torchtext.data.functional import to_map_style_dataset
train_split = dataset["train"]
print(len(train_split))
train_data = to_map_style_dataset(train_split)
len(train_data)

1072


1072

In [6]:
print(train_data[0]) 
print(train_data[0].keys())

{'tokens': ['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.'], 'pos_tags': ['ADP', 'DET', 'NOUN', 'DET', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'AUX', 'VERB', 'PUNCT'], 'ner_tags': ['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']}
dict_keys(['tokens', 'pos_tags', 'ner_tags'])


#Removing stopwords from the data


In [7]:
#Build a vocabulary from the data set
for i, token in enumerate(train_data):
  print(i)
  print(token)
  break

0
{'tokens': ['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.'], 'pos_tags': ['ADP', 'DET', 'NOUN', 'DET', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'AUX', 'VERB', 'PUNCT'], 'ner_tags': ['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']}
